In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

**read_data_small** is the function to read in the small dataset about 30 MB

In [2]:
def read_data_small():
    X_train = pd.read_csv("data/X_train.csv")
    X_test = pd.read_csv("data/X_test.csv")
    y_train = np.asarray(pd.read_csv("data/y_train.csv", header=None)[0])
    return X_train, X_test, y_train

In [4]:
pip install xgboost

     |████████████████████████████████| 829kB 5.6MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/prerakshah/opt/anaconda3/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/kq/02sx662n1ynfshzdhvmkp69c0000gn/T/pip-install-r09vgnhz/xgboost/setup.py'"'"'; __file__='"'"'/private/var/folders/kq/02sx662n1ynfshzdhvmkp69c0000gn/T/pip-install-r09vgnhz/xgboost/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: /private/var/folders/kq/02sx662n1ynfshzdhvmkp69c0000gn/T/pip-install-r09vgnhz/xgboost/
    Complete output (27 lines):
    ++ pwd
    + oldpath=/private/var/folders/kq/02sx662n1ynfshzdhvmkp69c0000gn/T/pip-install-r09vgnhz/xgboost
    + cd ./xgboost/
    + mkdir -p build
    + cd build
    + cmake ..
    ./xgboost/build-python.sh: line 21: cmake

# Insert Your Code Here

**detect_spoofying** is the function for training the classifier and classify the results. 

Here we provide an simple example.

In [3]:
### import libraries here ###
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import OneClassSVM
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
### code classifier here ###
def format_data(df):

    # append numberical columns
    rst = df.loc[:,["price","volume","bestBid","bestAsk",'bestBidVolume',
                    'bestAskVolume','lv2Bid', 'lv2BidVolume','lv2Ask', 
                    'lv2AskVolume', 'lv3Bid', 'lv3BidVolume', 'lv3Ask',
                    'lv3AskVolume']]
    
    # encode the binaries
    rst["isBid"] = df.isBid*1
    rst["isBuyer"] = df.isBuyer*1
    rst["isAggressor"] = df.isAggressor*1
    rst["type"] = (df.type == "ORDER")*1
    rst["source"] = (df.source=="USER")*1
    
    # parse the order id data
    rst["orderId"] = df.orderId.str.split('-').str[-1]
    rst["tradeId"] = df.tradeId.str.split('-').str[-1]
    rst["bidOrderId"] = df.bidOrderId.str.split('-').str[-1]
    rst["askOrderId"] = df.askOrderId.str.split('-').str[-1]
    
    # encode the multiple lable data
    tmp_operation = pd.DataFrame(pd.get_dummies(df.operation), columns=df.operation.unique()[:-1])
    rst = pd.concat([rst, tmp_operation], axis=1)
    tmp_endUserRef = pd.DataFrame(pd.get_dummies(df.endUserRef), columns=df.endUserRef.unique()[:-1])
    rst = pd.concat([rst, tmp_endUserRef], axis=1)
    
    return rst

def detect_spoofying(X_train, X_test, y_train):
    
    # clean up the data
    X_concat = pd.concat([X_train, X_test])
    X_clean = format_data(X_concat)
    
    # feel free to add more columns inferred from data
    # smartly engineered features can be very useful to improve the classification resutls
    X_clean["timeSinceLastTrade"] = pd.concat([X_train.loc[:,["timestamp","endUserRef"]].groupby("endUserRef").diff(),
                                              X_test.loc[:,["timestamp","endUserRef"]].groupby("endUserRef").diff()])
    
    X_clean = X_clean.fillna(-1)
    X_train_clean = X_clean.iloc[:X_train.shape[0],:]
    X_test_clean = X_clean.iloc[X_train.shape[0]:,:]
    X_train_clean_scaled = scale(X_train_clean)
    X_test_clean_scaled = scale(X_test_clean)

    # fit classifier
    clf = LogisticRegression(random_state=0, class_weight='balanced').fit(X_train_clean_scaled, y_train)
    y_train_prob_pred = clf.predict_proba(X_train_clean_scaled)
    y_test_prob_pred = clf.predict_proba(X_test_clean_scaled)

    #xgboost
    model = XGBClassifier().fit(X_train_clean_scaled, y_train,eval_metric="logloss")
    y_train_pred_x = model.predict(X_train_clean_scaled)
    y_test_pred_x = model.predict(X_test_clean_scaled)

    #naive bayes
    classifier = GaussianNB().fit(X_train_clean_scaled, y_train)
    y_train_pred_n = model.predict(X_train_clean_scaled)
    y_test_pred_n = classifier.predict(X_test_clean_scaled)

    #knn
    cl = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2).fit(X_train_clean_scaled, y_train)
    y_train_pred_k = model.predict(X_train_clean_scaled)
    y_test_pred_k = cl.predict(X_test_clean_scaled)

    #MLP
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(15, 2), random_state=1).fit(X_train_clean_scaled, y_train)
    y_train_mlp = clf.predict(X_train_clean_scaled)
    y_test_mlp = clf.predict(X_test_clean_scaled)

    return  y_train_prob_pred, y_test_prob_pred, y_train_pred_x,y_test_pred_x, y_train_pred_n, y_test_pred_n, y_train_pred_k,  y_test_pred_k, y_train_mlp, y_test_mlp, X_train_clean_scaled, X_test_clean_scaled,y_train




ModuleNotFoundError: No module named 'xgboost'

**score** is the function that we use to compare the results. An example is provided with scoring the predictions for the training dataset. True labels for the testing data set will be supplied to score the predictions for testing dataset.

Score is based on cohen's kappa measurement. https://en.wikipedia.org/wiki/Cohen%27s_kappa

In [0]:
from sklearn.metrics import cohen_kappa_score

def score(y_pred, y_true):
    """
    y_pred: a numpy 4d array of probabilities of point assigned to each label
    y_true: a numpy array of true labels
    """
    y_pred_label = np.argmax(y_pred, axis=1)
    return cohen_kappa_score(y_pred_label, y_true)

### Optional: k-fold cross validation

**wrapper** is the main function to read in unzipped data and output a score for evaluation. In addition, the function returns the y probability matrix (both train and test) for grading. More details about submitting format are outlined below.

In [0]:
def wrapper():
    # read in data
    X_train, X_test, y_train = read_data_small()
    # or if you have the computational power to work with the big data set, 
    # you can comment out the read_data_samll line and uncomment the following read_data_big
    # X_train, X_test, y_train = read_data_big()
    
    # process the data, train classifier and output probability matrix
    y_train_prob_pred, y_test_prob_pred, y_train_pred_x,y_test_pred_x, y_train_pred_n, y_test_pred_n, y_train_pred_k,  y_test_pred_k,y_train_mlp, y_test_mlp,X_train_clean_scaled,X_test_clean_scaled,y_train  = detect_spoofying(X_train, X_test, y_train)
    
    # return the prediciton values
    return  y_train_prob_pred, y_test_prob_pred, y_train_pred_x,y_test_pred_x, y_train_pred_n, y_test_pred_n, y_train_pred_k,  y_test_pred_k,y_train_mlp, y_test_mlp, X_train_clean_scaled, X_test_clean_scaled,y_train

Call function wrapper:

In [7]:
y_train_prob_pred, y_test_prob_pred, y_train_pred_x,y_test_pred_x, y_train_pred_n, y_test_pred_n, y_train_pred_k,  y_test_pred_k,y_train_mlp, y_test_mlp,X_train_clean_scaled,X_test_clean_scaled,y_train = wrapper()

KeyboardInterrupt: ignored

In [0]:
pd.DataFrame(y_test_pred_x).to_csv("logisticprediction.csv")

In [20]:
from sklearn.metrics import accuracy_score
print("Accuracy: {}".format(accuracy_score(y_train,y_train_pred_x)))
print("Accuracy: {}".format(accuracy_score(y_train,y_train_pred_n)))
print("Accuracy: {}".format(accuracy_score(y_train,y_train_pred_k)))
print("Accuracy: {}".format(accuracy_score(y_train,y_train_mlp)))

Accuracy: 0.9966577822904767
Accuracy: 0.9966577822904767
Accuracy: 0.9966577822904767
Accuracy: 0.9965661180916924


In [32]:
print(X_train_clean_scaled.shape)
print(X_test_clean_scaled.shape)
test = y_train.reshape((141822,1))
print(y_train.shape)

(141822, 27)
(60782, 27)
(141822,)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train_clean_scaled, y_train, test_size = 0.2)

In [34]:
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(113457, 27)
(28365, 27)
(113457,)
(28365,)


In [35]:
import keras
import keras.backend as K
from keras.layers import LSTM,Dropout,Dense,TimeDistributed,Conv1D,MaxPooling1D,Flatten
from keras.models import Sequential
import tensorflow as tf



Using TensorFlow backend.


In [0]:
n_signals = 1 #So far each instance is one signal. We will diversify them in next step
n_outputs = 4

In [37]:
verbose, epochs, batch_size = True, 1, 100
n_steps, n_length = 40, 10
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_signals)))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [38]:
def keras_auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras_auc])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.








In [39]:
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

ValueError: ignored